In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, Model, utils
import time
import numpy as np
data_dir = './data/cinic-10_image_classification_challenge-dataset/train/'

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
train_dataset = utils.image_dataset_from_directory(
    "./data/cinic-10_image_classification_challenge-dataset/train",
    image_size=(32, 32),
    label_mode="categorical",
    seed=123,
)

## Resnet gridsearch

In [ ]:
base_model101 = tf.keras.applications.ResNet101(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model101.layers:
  layer.trainable = False


base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False

In [ ]:
with open('results.txt', 'a') as file:
    for model, model_nr in zip([base_model50, base_model101], ['50', '101']):
        for optimize_algo in ['SGD', 'Adam', 'Adagrad']:
            for activation_func in ['relu', 'linear', 'softmax']:
                for regularization_algo, regularization_algo_name in zip([tf.keras.regularizers.L1, tf.keras.regularizers.L2], ['l1', 'l2']):
                    for l in [0.1, 0.3, 0.5]:

                        x = layers.Flatten()(model.output)
                        x = layers.Dense(1000, activation=activation_func, kernel_regularizer=regularization_algo(l))(x)
                        predictions = layers.Dense(10, activation = 'softmax')(x)

                        head_model = Model(inputs = model.input, outputs = predictions)
                        head_model.compile(optimizer=optimize_algo, loss="categorical_crossentropy", metrics=["accuracy"])

                        history = head_model.fit(train_dataset, epochs=3)
                        print(model_nr, optimize_algo, activation_func, regularization_algo_name, l, max(history.history['accuracy']))
                        file.write(model_nr + ' ' + optimize_algo + ' ' + activation_func + ' ' + regularization_algo_name + ' ' + str(l) + ' ' + str(max(history.history['accuracy'])) + '\n')

## show results

In [ ]:
import pandas as pd
pd.options.display.max_rows = 4000

data = pd.read_csv('results.txt', sep=' ', header=None)

data.sort_values(5, ascending=False)

## Continue training of pretrained resnet50 and resent101

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False

x = layers.Flatten()(base_model50.output)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation = 'softmax')(x)

head_model50 = Model(inputs = base_model50.input, outputs = predictions)
head_model50.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

history50 = head_model50.fit(train_dataset, epochs=50)

In [ ]:
base_model101 = tf.keras.applications.ResNet101(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model101.layers:
  layer.trainable = False

x = layers.Flatten()(base_model101.output)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation = 'softmax')(x)

head_model101 = Model(inputs = base_model101.input, outputs = predictions)
head_model101.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

history101 = head_model101.fit(train_dataset, epochs=50)

## Full training of ResNet50

In [ ]:
train_ds = utils.image_dataset_from_directory(
    './data/cinic-10_image_classification_challenge-dataset/train/',
    validation_split=0.2,
    subset='training',
    seed = 420,
    image_size=(32,32),
    batch_size=32,
    label_mode = 'categorical')

class_names = train_ds.class_names

val_ds = utils.image_dataset_from_directory(
    './data/cinic-10_image_classification_challenge-dataset/train/',
    validation_split=0.2,
    subset='validation',
    seed = 420,
    image_size=(32,32),
    batch_size=32,
    label_mode = 'categorical')

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = True

x = layers.Flatten()(base_model50.output)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation = 'softmax')(x)

head_model50 = Model(inputs = base_model50.input, outputs = predictions)
head_model50.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

history50 = head_model50.fit(train_ds, epochs=20)

In [ ]:
head_model50.save_weights("./weights/resnet50_full_training")